# Ceneo Scraper

## Extract  components for single opinion

|Component|Selector|Variable|
|---------|--------|--------|
|opinion ID |  ["data-entry-id"]  |opinion_id |
|opinion’s author |  span.user-post__author-name | author|
|author’s recommendation | span.user-post__author-recomendation > em |recommendations|
|score expressed in number of stars | span.user-post__score-count |score|
|opinion’s content | div.user-post__text |content|
|list of product advantages | div.review-feature__title--positives ~ div.review.-feature__items |pros|
|list of product disadvantages |div.review-feature__title--negatives ~ div.review.-feature__items |cons|
|how many users think that opinion was helpful | button.vote-yes > span |helpful|
|how many users think that opinion was unhelpful |button.vote-no > span | unhelpful|
|publishing date | span.user-post__published > time:nth-child(1)["datetime"]  |publish_date|
|purchase date | span.user-post__published > time:nth-child(2)["datetime"] |purchase_date|


# imports

In [68]:
import json
import os
import requests
from bs4 import BeautifulSoup

In [69]:
def extract_content(ancestor,selector,attribute=None,return_list=False):
            if selector:
                if return_list:
                    if attribute:
                        return [tag[attribute].strip() for tag in ancestor.select(selector)]
                    return [tag.text.strip() for tag in ancestor.select(selector)]
                if attribute:
                    try:
                        return ancestor.select_one(selector)[attribute].strip()
                    except TypeError:
                        return None
            try:
                return ancestor.select_one(selector).text.strip()
            except AttributeError:
                return None
            if attribute:
                  return ancestor[attribute]
            return ancestor[attribute]


## Opinion Structure

In [70]:
selectors={
"opinion_id":(None,'data-entry-id'),
"author":("span.user-post__author-name",),
"authors_recommendation":("span.user-post__author-recomendation > em",),
"stars" :("span.user-post__score-count",),
"content" :("div.user-post__text",),
"pros" :("div.review-feature__title--positives ~ div.review-feature__item",None,True),
"cons" :("div.review-feature__title--negatives ~ div.review-feature__item",None,True),
"helpful" :("button.vote-yes > span",),
"unhelpful" :("button.vote-no > span",),
"publish_date": ("span.user-post__published > time:nth-child(1)","datetime"),
"purchase_date":("span.user-post__published > time:nth-child(2)","datetime"),
}


In [72]:
def score(score:str) ->float:
    s=score.split("/")
    return float(s[0].replace(',','.'))/float(s[1])


## Dictionary 2

In [ ]:
transformations={
    "authors_recommendation": lambda r: True if r=="Polecam" else False if r== "nie polecam" else None
    "stars" : score:
    "helpful" :
    "unhelpful" :
}

# Send requests to Ceneo.pl service

In [ ]:

product_id= "108290707"
url=f"https://www.ceneo.pl/{product_id}"
response = requests.get(url)
response.status_code


200

In [ ]:
page_dom = BeautifulSoup(response.text,"html.parser")
opinions =page_dom.select("div.js_product-review")
all_opinions=[]

## Convert plain text HTML code into DOM structure

## Extract all components of single opinion

In [ ]:
all_opinions=[]
while(url):
  response = requests.get(url)
  response.status_code
  page_dom = BeautifulSoup(response.text,"html.parser")
  opinions =page_dom.select("div.js_product-review")
  for opinion in opinions:
    single_opinion={
        key: extract_content(opinion, *value)
          for key,value in selectors.items()
    }
    for key,value in transformations.item():
      single.opinion[key]=value(single.opinion[key])
      
    all_opinions.append(single_opinion)
  try:
    url ="https://www.ceneo.pl"+extract_content(page_dom,"a.pagination__next","href")
  except TypeError:
    url=None

In [ ]:
if not os.path.exists("opinions"):
    os.mkdir("opinions")
    
with open(f"opinions/{product_id}.json","w",encoding="UTF-8") as jf:
    json.dump(all_opinions,jf,indent=4,ensure_ascii=False)
 

In [ ]:
len(all_opinions)

45